In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
pip install python-terrier==0.10.0 nltk scikit-learn lightgbm xgboost fastrank

Note: you may need to restart the kernel to use updated packages.


In [3]:
# !pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR

In [4]:
import pandas as pd
import json
import pyterrier as pt
import random

/home/roberto/delft/inforet/moment_retrieval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#UNCOMMENT THE FOLLOWING LINE TO USE EITHER THE TVR DATASET OR THE QVH DATASET

dataset_choice = "TVR"
# dataset_choice = "QVH"

In [6]:
# Paths to JSONL files on Colab
# if dataset_choice == "TVR":
#    jsonl_train_path = '/content/drive/MyDrive/IR/text_data/tvr_train_release.jsonl'
#    jsonl_val_path = '/content/drive/MyDrive/IR/text_data/tvr_val_release.jsonl'
#    subs_path = '/content/drive/MyDrive/IR/text_data/tvqa_preprocessed_subtitles.jsonl'

# elif dataset_choice == "QVH":
#    jsonl_train_path = "/content/drive/MyDrive/IR/text_data_QVH/highlight_train_release.jsonl"
#    subs_path = "/content/drive/MyDrive/IR/text_data_QVH/subs_train.jsonl"

In [7]:
# Paths to JSONL files
if dataset_choice == "TVR":
    jsonl_train_path = 'text_data/tvr_train_release.jsonl'
    jsonl_val_path = 'text_data/tvr_val_release.jsonl'
    subs_path = 'text_data/tvqa_preprocessed_subtitles.jsonl'

elif dataset_choice == "QVH":
    jsonl_train_path = "text_data_QVH/highlight_train_release.jsonl"
    subs_path = "text_data_QVH/subs_train.jsonl"

In [8]:
# Load subtitles into a dictionary for quick access
subtitles_dict = {}
if dataset_choice == "TVR":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            subtitles_dict[sub_data['vid_name']] = sub_data['sub']
elif dataset_choice == "QVH":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            triple = sub_data['vid'].split("_")
            name = sub_data['vid']
            #turn the list name into a string
            name = "".join(name)
            if name not in subtitles_dict:
                subtitles_dict[name] = [(float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query'])]
            else:
                subtitles_dict[name].append((float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query']))
            
            # print(subtitles_dict[name])


In [9]:
# Function to find matching subtitles in TVR case
def find_matching_subtitles(vid_name, ts_range, subtitles_dict):
    matching_subs = []
    if vid_name in subtitles_dict:
        for subtitle in subtitles_dict[vid_name]:
            if (ts_range[0] <= subtitle['start'] <= ts_range[1]) or (ts_range[0] <= subtitle['end'] <= ts_range[1]) or (subtitle['start'] <= ts_range[0] and subtitle['end'] >= ts_range[1]):
                matching_subs.append(subtitle['text'])
    return matching_subs

In [10]:
def parse_jsonl_TVR(jsonl_path, split_type):
    # Initialize empty lists for your data
    queries_data = []
    documents_data = []
    query_rankings_data = []

    with open(jsonl_path, 'r') as file:
        for idx, line in enumerate(file):
            data = json.loads(line)
            # drop non text-based queries
            if data['type'] not in ['t']:
                continue

            # Find matching subtitles
            matching_subs = find_matching_subtitles(data['vid_name'], data['ts'], subtitles_dict)

            if matching_subs == []:
                continue

            # Extract data for the Query Set DataFrame
            queries_data.append({'qid': str(data['desc_id']), 'query': data['desc']})

            # Extract data for the Documents Set DataFrame, including matching subtitles
            documents_data.append({'docno': split_type + str(idx), 'vid_name': data['vid_name'], 'ts': data['ts'],
                                'duration': data['duration'], 'type': data['type'], 'text': "".join(matching_subs)})

            # Extract data for the Query Rankings DataFrame
            query_rankings_data.append({'qid': str(data["desc_id"]), 'query': data['desc'], 'docno': split_type + str(idx), 'rank': 1, 'score': 1.0})

    return queries_data, documents_data, query_rankings_data

def parse_jsonl_QVH(jsonl_path):
    queries_data = []
    documents_data = []
    query_rankings_data = []
    with open(jsonl_path, 'r') as file:
        for idx,line in enumerate(file):

            # Load the JSON object from the line
            data = json.loads(line)

            triple = data["vid"].split("_")
            document_name = triple[0:-2]
            document_name = data["vid"]
            start_time = float(triple[-2])
            end_time = float(triple[-1])

            if document_name not in subtitles_dict:
                #print("Document not found in subtitles: ", document_name)
                continue
            
            subs = []
            for relevant_window in data["relevant_windows"]:
                ts = [start_time+relevant_window[0], start_time+relevant_window[1]]
                subs.extend([sub for sub in subtitles_dict[document_name] if sub[0] <= ts[1] and ts[0] <= sub[1]])
                if len(subs) == 0:
                    #print("No subtitles found for ", document_name, " at time ", ts)
                    continue
            
            documents_data.append({"docno": str(idx), "vid_name": document_name, "ts": [start_time, end_time], "duration" : data["duration"], "text": " ".join([sub[2] for sub in subs])})
            queries_data.append({"qid" : str(data["qid"]), "query": data["query"]})
            query_rankings_data.append({"qid": str(data["qid"]), "query": data["query"], "docno": str(idx), "rank": 1, "score": 1.0})

    return queries_data, documents_data, query_rankings_data


In [11]:

if dataset_choice == "TVR":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_TVR(jsonl_train_path, "t")
    queries_data_val, documents_data_val, query_rankings_data_val = parse_jsonl_TVR(jsonl_val_path, "v")
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]

elif dataset_choice == "QVH":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_QVH(jsonl_train_path)
    #have to create a val set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_val = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_val]
    queries_data_val = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_val]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_val]
    documents_data_val = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_val]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_val]
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]



In [12]:
# Create DataFrames for the Query Set, Documents Set, and Query Rankings
queries_train_df = pd.DataFrame(queries_data_train)
documents_train_df = pd.DataFrame(documents_data_train)

queries_val_df = pd.DataFrame(queries_data_val)
documents_val_df = pd.DataFrame(documents_data_val)

queries_test_df = pd.DataFrame(queries_data_test)
documents_test_df = pd.DataFrame(documents_data_test)

q_rels = pd.concat([pd.DataFrame(query_rankings_data_train), pd.DataFrame(query_rankings_data_val), pd.DataFrame(query_rankings_data_test)]).reset_index(drop=True)

#print length of the dataframes
print("Train set:")
print("Queries: ", len(queries_train_df))
print("Documents: ", len(documents_train_df))

print("Val set:")
print("Queries: ", len(queries_val_df))
print("Documents: ", len(documents_val_df))

print("Test set:")
print("Queries: ", len(queries_test_df))
print("Documents: ", len(documents_test_df))

print("Query Rankings: ", len(q_rels))



Train set:
Queries:  7110
Documents:  7110
Val set:
Queries:  957
Documents:  957
Test set:
Queries:  790
Documents:  790
Query Rankings:  8857


### First Stage Retrieval [TODO: BOX]
The following part of the code will define three different first stage retrieval pipelines as an input for the trained model.

In [13]:
!ls -la /usr/lib/jvm/java-11-openjdk-amd64
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

if not pt.started():
    pt.init()

total 32
drwxr-xr-x  7 root root 4096 feb 29 19:29 .
drwxr-xr-x  4 root root 4096 feb 29 19:29 ..
drwxr-xr-x  2 root root 4096 feb 29 19:29 bin
drwxr-xr-x  4 root root 4096 feb 29 19:29 conf
lrwxrwxrwx  1 root root   42 mag  9  2023 docs -> ../../../share/doc/openjdk-11-jre-headless
drwxr-xr-x 73 root root 4096 giu  9  2023 legal
drwxr-xr-x  7 root root 4096 feb 29 19:29 lib
drwxr-xr-x  4 root root 4096 giu  9  2023 man
-rw-r--r--  1 root root 1250 gen 20 11:27 release


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [14]:
# Create an index
from pathlib import Path

indexer = pt.IterDictIndexer(
    "./index_path/",
    meta={
        "docno": 64,
        "vid_name": 64,
        "text": 131072,
    },
    stemmer="porter",
    stopwords="terrier",
    overwrite=True,
    # type=pt.index.IndexingType.MEMORY,
)

In [15]:
joint_documents_set_df = pd.concat([documents_train_df, documents_val_df, documents_test_df])

print("Length: ", len(joint_documents_set_df))

Length:  8857


In [16]:
indexed = indexer.index(
    joint_documents_set_df.to_dict(orient="records")
)

13:35:18.507 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (t24499) - further warnings are suppressed
13:35:20.426 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


In [17]:
#These are some examples, not necessarily the best ones. Experiment with different models
num_res = 10000

# Initialize BatchRetrieve with the created index and specify BM25 as the weighting model
first_stage_bm25 = pt.BatchRetrieve(
    indexed,
    wmodel="BM25",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)


# Initialize BatchRetrieve with the created index and specify LemurTF_IDF as the weighting model
first_stage_lemurtfidf = pt.BatchRetrieve(
    indexed,
    wmodel="LemurTF_IDF",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify Hiemstra_LM as the weighting model
first_stage_hiemstra_lm = pt.BatchRetrieve(
    indexed,
    wmodel="Hiemstra_LM",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_dlh = pt.BatchRetrieve(
    indexed,
    wmodel="DLH",
    num_results=num_res,
    metadata=['docno', 'vid_name', 'text']
)

first_stage_dfic = pt.BatchRetrieve(
    indexed,
    wmodel="DFIC",
    num_results=num_res,
    metadata=['docno', 'vid_name', 'text']
)

first_stage_lgd = pt.BatchRetrieve(
    indexed,
    wmodel="LGD",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_in_exp_b2 = pt.BatchRetrieve(
    indexed,
    wmodel="In_expB2",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_dirchlet = pt.BatchRetrieve(
    indexed,
    wmodel="DirichletLM",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_coord = pt.BatchRetrieve(
    indexed,
    wmodel="CoordinateMatch",
    num_results=num_res,
    metadata=["docno", "vid_name", "text"]
)

first_stage_coord_qe = pt.BatchRetrieve(
    indexed,
    wmodel="CoordinateMatch",
    num_results=num_res,
    controls={"qe":"on", "qemodel" : "Bo1"},
    metadata=["docno", "vid_name", "text"]
)

first_stage_pl2 = pt.BatchRetrieve(
    indexed,
    wmodel="PL2",
    num_results=num_res,
    metadata=['docno']
)

bm25_qe = pt.BatchRetrieve(indexed, wmodel="BM25", controls={"qe":"on", "qemodel" : "Bo1"}, num_results=num_res)

# Computing feature
The weighting model can be use in [pt.weighting_model.package: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html)

In [18]:
#We create features for the second stage using the first stage retrievers

#TF-IDF based features
lemur_tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="LemurTF_IDF")
bm25_retriever = pt.BatchRetrieve(indexed, wmodel="BM25")
tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="TF_IDF")

#Language model based features
hiem_retriever = pt.BatchRetrieve(indexed, wmodel="Hiemstra_LM")
dirichlet_retriever = pt.BatchRetrieve(indexed, wmodel="DirichletLM")

#Divergence from randomness based features
pl2_retriever = pt.BatchRetrieve(indexed, wmodel="PL2")
dph_retriever = pt.BatchRetrieve(indexed, wmodel="DPH")
dlh_retriever = pt.BatchRetrieve(indexed, wmodel="DLH")

#Can add more!
coordinate_match_retriever = pt.BatchRetrieve(indexed, wmodel="CoordinateMatch")
js_kls_retrieveer = pt.BatchRetrieve(indexed, wmodel="Js_KLs")


In [19]:
# PIPELINES WITH FSR AND FEATURES

#Can test many possibilities, i just put some examples here

bm25_pipeline = ~first_stage_bm25 >> (
   tf_idf_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer ** bm25_retriever
)


lemurtf_idf_pipeline = ~first_stage_lemurtfidf >> (
    pl2_retriever ** dph_retriever ** hiem_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer
)

hiem_lm_pipeline = ~hiem_retriever >> (
    pl2_retriever ** dph_retriever ** tf_idf_retriever ** coordinate_match_retriever ** dirichlet_retriever ** js_kls_retrieveer
)

#NOTE: i believe we should not use the same first stage retriever for features AND for the first stage retrieval

In [20]:
# Prepare the queries for the pipeline, remove special characters and extra spaces
prepared_trainqueries = queries_train_df
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels = pd.DataFrame(query_rankings_data_train)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels['label'] = prepared_train_qrels['score']

prepared_train_qrels['label'] = prepared_train_qrels['label'].astype(int)

prepared_val_qrels = pd.DataFrame(query_rankings_data_val)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_val_qrels['label'] = prepared_val_qrels['score']

prepared_val_qrels['label'] = prepared_val_qrels['label'].astype(int)

prepared_test_qrels = pd.DataFrame(query_rankings_data_test)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_test_qrels['label'] = prepared_test_qrels['score']

prepared_test_qrels['label'] = prepared_test_qrels['label'].astype(int)

prepared_qrels = q_rels
prepared_qrels['query'] = prepared_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_qrels['label'] = prepared_qrels['score']

prepared_qrels['label'] = prepared_qrels['label'].astype(int)

prepared_valqueries = queries_val_df.reset_index()
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)

prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

#test set
prepared_testqueries = queries_test_df.reset_index()
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()


prepared_queries = pd.concat([prepared_trainqueries, prepared_valqueries, prepared_testqueries]).reset_index(drop=True)
prepared_qrels = pd.concat([prepared_train_qrels, prepared_val_qrels, prepared_test_qrels]).reset_index(drop=True)
prepared_documents = pd.concat([documents_train_df, documents_val_df, documents_test_df]).reset_index(drop=True)


Storage format: .csv or trec

trec(https://pyterrier.readthedocs.io/en/latest/io.html): The pt io format, but it doesn't contain feature.

In [21]:
from sklearn.svm import SVR
import xgboost as xgb
import fastrank

index = pt.IndexFactory.of("./index_path")

fsr_pipelines = [
    #{
    #    'pipe': lemurtf_idf_pipeline,
    #    'name': 'LemurTF_IDF'
    #},
    #{
    #    'pipe': bm25_pipeline,
    #    'name': 'BM25'
    #},
    #{
    #    'pipe': hiem_lm_pipeline,
    #    'name': 'Hiemstra LM'
    #}
]

learned_models = [
    # {
    #     'model': SVR(),
    #     'form': 'reg',
    #     'name': 'SVR',
    # },
    {
        'model': xgb.XGBRanker(tree_method="hist", objective="rank:ndcg"),
        'form': 'ltr',
        'name': 'XGBoost (NDCG)',
    },
    #{
    #    'model': xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:pairwise", lambdarank_pair_method="topk"),
    #    'form': 'ltr',
    #    'name': 'XGBoost (Pariwise)'
    #}
    # {
    #     'model': xgb.XGBRanker(tree_method="hist", objective="rank:map"),
    #     'form': 'ltr',
    #     'name': 'XGBoost (MAP)',
    # },
    #{
    #    'model': fastrank.TrainRequest.coordinate_ascent(),
    #    'form': 'fastrank',
    #    'name': 'FastRank Coordinate Ascent',
    #},
    #{
    #    'model': fastrank.TrainRequest.random_forest(),
    #    'form': 'fastrank',
    #    'name': 'FastRank Random Forest',
    #}
  ]

In [23]:
from pyterrier.measures import *

trained_models = [first_stage_bm25, bm25_qe, first_stage_lemurtfidf,first_stage_pl2,first_stage_dlh, first_stage_hiemstra_lm, first_stage_dirchlet,
                  first_stage_dfic,  first_stage_lgd, first_stage_in_exp_b2,
                  first_stage_coord, first_stage_coord_qe ]
names = ['BM25',  'BM25 + QE', 'LemurTF_IDF', 'PL2','DLH', 'Hiemstra LM', 'Dirichlet LM',
         'DFIC', 'LGD', 'In Exp  B2', 
         'CoordinateMatch', 'CoordinateMatch + QE',
        ]
eval_metrics = [
    MAP, NumRelRet, 
    Recall @ 1,   Recall @ 3,    Recall @ 5,    Recall @ 10, 
    Recall @ 20,  Recall @ 30,   Recall @ 50,   Recall @ 100,
    Recall @ 200, Recall @ 300,  Recall @ 500,  Recall @ 1000, 
    Recall @ 2000,Recall @ 3000, Recall @ 5000, Recall @ 10000 
]

for fsr in fsr_pipelines:
    for model in learned_models:
        names.append(f"{fsr['name']} >> {model['name']}")
        print(names[-1])
        if 'form' in model:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'], form=model['form'])
        else:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'])
        pipe.fit(
            prepared_trainqueries,
            prepared_train_qrels,
            prepared_valqueries,
            prepared_val_qrels
        )
        trained_models.append(pipe)

In [24]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_testqueries,
    prepared_test_qrels,
    names=names,
    eval_metrics=eval_metrics,
)

,name,AP,NumRet(rel=1),R@1,R@3,R@5,R@10,R@20,R@30,R@50,R@100,R@200,R@300,R@500,R@1000,R@2000,R@3000,R@5000,R@10000
0,BM25,0.250901,699.0,0.193671,0.270886,0.301266,0.355696,0.407595,0.440506,0.489873,0.572152,0.670886,0.725316,0.788608,0.860759,0.883544,0.884810,0.884810,0.884810
1,BM25 + QE,0.245801,725.0,0.182278,0.273418,0.300000,0.354430,0.402532,0.448101,0.489873,0.572152,0.678481,0.732911,0.794937,0.870886,0.905063,0.913924,0.917722,0.917722
2,LemurTF_IDF,0.232264,699.0,0.175949,0.253165,0.287342,0.330380,0.387342,0.421519,0.467089,0.543038,0.639241,0.708861,0.781013,0.848101,0.881013,0.884810,0.884810,0.884810
3,PL2,0.240868,699.0,0.178481,0.267089,0.294937,0.350633,0.405063,0.435443,0.478481,0.563291,0.650633,0.717722,0.787342,0.858228,0.883544,0.884810,0.884810,0.884810
4,DLH,0.239276,699.0,0.183544,0.260759,0.287342,0.343038,0.383544,0.417722,0.473418,0.555696,0.641772,0.691139,0.774684,0.853165,0.881013,0.884810,0.884810,0.884810
5,Hiemstra LM,0.236868,699.0,0.183544,0.255696,0.283544,0.331646,0.391139,0.410127,0.459494,0.546835,0.627848,0.687342,0.770886,0.851899,0.883544,0.884810,0.884810,0.884810
6,Dirichlet LM,0.165745,699.0,0.115190,0.182278,0.217722,0.251899,0.286076,0.303797,0.343038,0.426582,0.536709,0.618987,0.706329,0.824051,0.878481,0.884810,0.884810,0.884810
7,DFIC,0.232726,699.0,0.179747,0.241772,0.272152,0.340506,0.387342,0.417722,0.472152,0.555696,0.640506,0.701266,0.781013,0.854430,0.883544,0.884810,0.884810,0.884810
8,LGD,0.229390,699.0,0.173418,0.243038,0.282278,0.334177,0.383544,0.421519,0.470886,0.553165,0.646835,0.703797,0.782278,0.860759,0.884810,0.884810,0.884810,0.884810
9,In Exp B2,0.254581,699.0,0.192405,0.282278,0.313924,0.364557,0.416456,0.456962,0.505063,0.587342,0.697468,0.750633,0.803797,0.867089,0.883544,0.884810,0.884810,0.884810


In [25]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_valqueries,
    prepared_val_qrels,
    names=names,
    eval_metrics=eval_metrics,
)

,name,AP,NumRet(rel=1),R@1,R@3,R@5,R@10,R@20,R@30,R@50,R@100,R@200,R@300,R@500,R@1000,R@2000,R@3000,R@5000,R@10000
0,BM25,0.262509,852.0,0.196447,0.281087,0.324974,0.400209,0.458725,0.490073,0.538140,0.599791,0.681296,0.739812,0.807732,0.873563,0.890282,0.890282,0.890282,0.890282
1,BM25 + QE,0.254552,887.0,0.179728,0.284222,0.335423,0.393939,0.449321,0.484848,0.539185,0.608150,0.694880,0.759666,0.817137,0.888192,0.913271,0.926855,0.926855,0.926855
2,LemurTF_IDF,0.236941,852.0,0.167189,0.270637,0.310345,0.354232,0.412748,0.452456,0.506792,0.572623,0.669801,0.718913,0.793103,0.865204,0.890282,0.890282,0.890282,0.890282
3,PL2,0.255094,852.0,0.190178,0.273772,0.317659,0.385580,0.449321,0.482759,0.528736,0.592476,0.676071,0.732497,0.797283,0.868339,0.890282,0.890282,0.890282,0.890282
4,DLH,0.243870,852.0,0.176594,0.266458,0.296761,0.368861,0.442006,0.470219,0.516196,0.578892,0.665622,0.718913,0.793103,0.863114,0.888192,0.890282,0.890282,0.890282
5,Hiemstra LM,0.241322,852.0,0.179728,0.265413,0.303030,0.357367,0.416928,0.453501,0.493208,0.563218,0.654127,0.716823,0.793103,0.863114,0.889237,0.890282,0.890282,0.890282
6,Dirichlet LM,0.153291,852.0,0.095089,0.169279,0.215256,0.261233,0.309300,0.338558,0.382445,0.474399,0.595611,0.650993,0.734587,0.836991,0.889237,0.890282,0.890282,0.890282
7,DFIC,0.238504,852.0,0.174504,0.257053,0.291536,0.352142,0.434692,0.475444,0.510972,0.582027,0.664577,0.721003,0.789969,0.868339,0.889237,0.890282,0.890282,0.890282
8,LGD,0.241330,852.0,0.179728,0.257053,0.290491,0.350052,0.435737,0.474399,0.517241,0.577847,0.663532,0.724138,0.798328,0.873563,0.890282,0.890282,0.890282,0.890282
9,In Exp B2,0.263930,852.0,0.187043,0.304075,0.337513,0.402299,0.455590,0.496343,0.551724,0.622780,0.706374,0.764890,0.832811,0.879833,0.890282,0.890282,0.890282,0.890282
